In [ ]:
(ql:quickload :cytoscape-clj)

In [ ]:
(defparameter *count* 0)
(defparameter +size+ 300)
(defparameter +margin+ 30)
(defparameter +columns+ 4)

(defparameter *cyto* (cytoscape:make-cytoscape-widget
                       :layout (make-instance 'jupyter-widgets:layout :height "640px")
                       :graph-style "node { content: data(name); } 
                                                  node.child { text-valign: center; text-halign: center; }
                                                  edge { curve-style: bezier; }"))

(defun element-id (id i)
  (format nil "~A-~A" id i))

(defun add-graphs (&rest layouts)
  (dolist (layout layouts)
    (let ((id (jupyter:make-uuid))
          (elements nil))
      (push (cytoscape:make-element
              :group "nodes"
              :data (j:make-object "id" id
                                   "name" (string-downcase (class-name (class-of layout))))
              :classes (list id))
            elements)
      (dotimes (i 7)
        (push (cytoscape:make-element
                :group "nodes"
                :data (j:make-object "id" (element-id id i)
                                     "parent" id
                                     "name" (format nil "~A" i))
                :classes (list id "child"))
              elements))
      (dotimes (i 3)
        ; Edge from the corner to the next side
        (push (cytoscape:make-element
                :group "edges"
                :data (j:make-object "parent" id
                                     "id" (jupyter:make-uuid)
                                     "source" (element-id id (* 2 i))
                                     "target" (element-id id (1+ (* 2 i))))
                :classes (list id))
              elements)
        ; Edge from the corner to the previous side
        (push (cytoscape:make-element
                :group "edges"
                :data (j:make-object "parent" id
                                     "id" (jupyter:make-uuid)
                                     "source" (element-id id (* 2 i))
                                     "target" (element-id id (mod (1- (* 2 i)) 6)))
                :classes (list id))
              elements)
        ; Edge from the corner to the middle
        (push (cytoscape:make-element
                :group "edges"
                :data (j:make-object "parent" id
                                     "id" (jupyter:make-uuid)
                                     "source" (element-id id (* 2 i))
                                     "target" (element-id id 6))
                :classes (list id))
              elements)
        ; Edge from the side to the next side
        (push (cytoscape:make-element
                :group "edges"
                :data (j:make-object "parent" id
                                     "id" (jupyter:make-uuid)
                                     "source" (element-id id (1+ (* 2 i)))
                                     "target" (element-id id (mod (1- (* 2 i)) 6)))
                :classes (list id))
              elements)
        ; Edge from the side to the middle
        (push (cytoscape:make-element
                :group "edges"
                :data (j:make-object "parent" id
                                     "id" (jupyter:make-uuid)
                                     "source" (element-id id (1+ (* 2 i)))
                                     "target" (element-id id 6))
                :classes (list id))
              elements))
      (setf (cytoscape:selector layout) (format nil ".~A" id))
      (multiple-value-bind (row col) (floor *count* +columns+)
        (setf (cytoscape:bounding-box layout) (list (cons "x1" (* (+ +margin+ +size+) col))
                                                    (cons "y1" (* (+ +margin+ +size+) row))
                                                    (cons "w" +size+)
                                                    (cons "h" +size+))))
      (incf *count*)
      (setf (cytoscape:fit layout) nil)
      (cytoscape:on-layout-stop layout (lambda (l) (declare (ignore l)) (cytoscape:fit-elements *cyto*)))
      (setf (cytoscape:graph-layouts *cyto*) (append (cytoscape:graph-layouts *cyto*) (list layout)))
      (setf (cytoscape:elements *cyto*) (append (cytoscape:elements *cyto*) (reverse elements)))))
  (values))

*cyto*

In [ ]:
(add-graphs (cytoscape:make-breadth-first-layout)
            (cytoscape:make-random-layout)
            (cytoscape:make-grid-layout)
            (cytoscape:make-circle-layout)
            (cytoscape:make-concentric-layout)
            (cytoscape:make-cose-layout)
            (cytoscape:make-cola-layout)
            (cytoscape:make-dagre-layout))